In [71]:
import pyxdf
import mne
import numpy as np
import pandas as pd
import random

In [72]:
# streams มี 2 ช่องที่เป็นข้อมูลเข้ามาคือ 1. time series ที่มาจาก openBCI 2. ข้อมูลของ maekers
streams, header = pyxdf.load_xdf("C:/Users/Toey/Desktop/data_ssvep/Toey/exp1/block_2.xdf")

In [73]:
raw_data = streams[0]["time_series"].T #From Steam variable this query is EEG data
print(raw_data.shape)

(8, 229960)


In [74]:
channels =['Fz','Cz','P4','Pz','P3','P08','Oz','P07'] #Set your target EEG channel name
info = mne.create_info(
    ch_names= channels,
    ch_types= ['eeg']*len(channels),
    sfreq= 250 #OpenBCI Frequency acquistion
)
# Create MNE rawarray
raw_mne = mne.io.RawArray(raw_data, info, verbose=False)
print(raw_mne)

<RawArray | 8 x 229960 (919.8 s), ~14.0 MB, data loaded>


In [75]:
event_index = streams[1]["time_series"].T[0] #Get all event marker of experiment 
#.T คือ transpose array [0] data ใน column 0 ของ streams[1]["time_series"]
print(event_index.shape)

(180,)


In [76]:
event_timestamp = streams[1]["time_stamps"].T #Timestamp when event marked
# รับเวลาที่ marker นั้นอยู่
print(event_timestamp.shape)

(180,)


In [77]:
events = np.column_stack((np.array(event_timestamp, dtype = int),
                        np.zeros(len(event_timestamp), dtype = int),
                        np.array(event_index, dtype = int)))
fre1 = 8
fre2 = 10
fre3 = 12
fre4 = 14
events_id = { # Set up your event name
     f'natural': 0,
     f'Frequency {fre1} Hz': 1,
     f'Frequency {fre2} Hz': 2,
     f'Frequency {fre3} Hz': 3,
     f'Frequency {fre4} Hz': 4,
}
mne_epochs = mne.Epochs(raw_mne, events, 
        tmin= -5,     # init timestamp of epoch (0 means trigger timestamp same as event start)
        tmax= 10,    # final timestamp (10 means set epoch duration 10 second)
        event_id =events_id,
        preload = True,
        event_repeated='drop',
        baseline=(-0.5, 0)
    )

Not setting metadata
180 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 180 events and 3751 original time points ...
0 bad epochs dropped


In [78]:
print(mne_epochs[138].event_id)

{'natural': 0}


In [79]:
# สร้าง list เพื่อเก็บ index ของ epoch ที่ต้องการลบ event_id ออก
def count_each_class(mne_epochs, num_class):
    indices_to_drop = []
    num_class = num_class

    # Iterate ผ่านทุกๆ epoch
    for idx, epoch in enumerate(mne_epochs):
        # ตรวจสอบว่า event_id เป็น {'natural': 0} หรือไม่
        if mne_epochs.events[idx, -1] == num_class:  # ตรวจสอบ event_id ในข้อมูล events ของ epoch นั้นๆ
            # เพิ่ม index ของ epoch ที่ต้องการลบออกใน list
            indices_to_drop.append(idx)

    # ลบ epoch ที่มี event_id เป็น {'natural': 0} ออกจาก mne_epochs
    return indices_to_drop
    # mne_epochs.drop(indices_to_drop)


In [80]:
import random

def random_drop_to_balance(mne_epochs, class_counts):
    # หาจำนวนของ class ที่มีจำนวนน้อยสุด
    # print(class_counts)
    min_count = min(class_counts)
    
    # สร้าง list เพื่อเก็บ index ของแต่ละ class
    class_indices = [[] for _ in range(len(class_counts))]

    # นับ index ของแต่ละ class
    for idx, epoch in enumerate(mne_epochs):
        event_id = mne_epochs.events[idx, -1]
        class_indices[event_id].append(idx)

    # สุ่มลำดับ index ของแต่ละ class เพื่อทำการลบ
    random_indices_to_drop = []
    for indices in class_indices:
        # สุ่มลำดับ index ที่ต้องการลบออก
        if len(indices) > len(min_count):
            random_indices = random.sample(indices, len(indices) - len(min_count))
            random_indices_to_drop.extend(random_indices)
    
    # ลบ epoch ที่ต้องการออกจาก mne_epochs
    mne_epochs.drop(random_indices_to_drop)

    return random_indices_to_drop

# ใช้ฟังก์ชันเพื่อสุ่มลบ index ของแต่ละ class เพื่อให้จำนวนของแต่ละ class เท่ากับ class ที่มีจำนวนน้อยสุด
class0 = count_each_class(mne_epochs, 0)
class1 = count_each_class(mne_epochs, 1)
class2 = count_each_class(mne_epochs, 2)
class3 = count_each_class(mne_epochs, 3)
class4 = count_each_class(mne_epochs, 4)

class_counts = [class0, class1, class2, class3, class4]
random_indices_to_drop = random_drop_to_balance(mne_epochs, class_counts)
# print(random_indices_to_drop)


Dropped 80 epochs: 1, 3, 5, 7, 8, 12, 14, 16, 17, 19, 21, 25, 26, 30, 34, 35, 39, 41, 44, 48, 50, 52, 53, 55, 57, 59, 61, 62, 64, 66, 68, 70, 71, 77, 79, 80, 82, 84, 89, 91, 93, 95, 96, 97, 98, 100, 102, 104, 106, 107, 109, 111, 113, 115, 116, 120, 122, 125, 127, 129, 131, 133, 136, 138, 143, 147, 149, 151, 152, 154, 156, 158, 160, 161, 165, 169, 170, 172, 174, 176


In [81]:
mne_epochs

Number of events,100
Events,Frequency 10 Hz: 20Frequency 12 Hz: 20Frequency 14 Hz: 20Frequency 8 Hz: 20natural: 20
Time range,-5.000 – 10.000 s
Baseline,-0.500 – 0.000 s


In [82]:
mne_epochs.get_data().shape

C:\Users\Toey\AppData\Local\Temp\ipykernel_15276\1290654083.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  mne_epochs.get_data().shape


(100, 8, 3751)

In [83]:
# dataset = mne_epochs.to_data_frame()
# id_df = pd.DataFrame({"id": range(len(dataset))})
# dataset = pd.concat([id_df,dataset], axis=1)
# dataset

,id,time,condition,epoch,Fz,Cz,P4,Pz,P3,P08,Oz,P07
0,0,-5.000,Frequency 8 Hz,0,-6.644051e+08,3.412238e+08,-7.028111e+09,-8.324441e+08,3.788887e+09,-7.457960e+08,-8.307119e+08,-8.082290e+08
1,1,-4.996,Frequency 8 Hz,0,-6.933474e+07,1.521506e+08,-6.590861e+09,-1.198258e+08,8.993210e+08,-9.189562e+07,-9.270165e+07,-9.038038e+07
2,2,-4.992,Frequency 8 Hz,0,6.079455e+08,-1.332811e+08,3.015139e+09,6.935095e+08,-2.930002e+09,6.879120e+08,7.356760e+08,7.155791e+08
3,3,-4.988,Frequency 8 Hz,0,4.591057e+08,-1.257489e+08,7.605389e+09,5.220940e+08,-2.466070e+09,5.432064e+08,5.516987e+08,5.384414e+08
4,4,-4.984,Frequency 8 Hz,0,-1.531980e+08,1.238527e+08,2.708889e+09,-2.141501e+08,8.791149e+08,-1.544806e+08,-2.062710e+08,-1.936675e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
375095,375095,9.984,Frequency 14 Hz,179,-3.079675e+08,4.269578e+06,2.765746e+09,-2.027933e+08,8.387241e+08,-1.627788e+08,-2.201881e+08,-1.990811e+08
375096,375096,9.988,Frequency 14 Hz,179,-8.165808e+08,2.326153e+08,-7.023254e+09,-8.166839e+08,3.834215e+09,-7.631470e+08,-8.438692e+08,-8.059981e+08
375097,375097,9.992,Frequency 14 Hz,179,-2.148278e+08,3.086821e+07,-6.564004e+09,-9.615312e+07,8.458989e+08,-1.012002e+08,-9.904161e+07,-8.365675e+07
375098,375098,9.996,Frequency 14 Hz,179,4.756185e+08,-2.818320e+08,3.231996e+09,7.324260e+08,-3.153096e+09,6.994844e+08,7.481786e+08,7.374126e+08


In [84]:
# dataset.groupby('condition').size()

condition
Frequency 10 Hz    75020
Frequency 12 Hz    75020
Frequency 14 Hz    75020
Frequency 8 Hz     75020
natural            75020
dtype: int64

In [96]:
# feature_columns = ['Fz','Cz','P4','Pz','P3','P08','Oz','P07']
# x = dataset[feature_columns].values
# y = dataset['condition'].values

# # print(x)

[[-6.64405056e+08  3.41223803e+08 -7.02811111e+09 ... -7.45796015e+08
  -8.30711908e+08 -8.08229016e+08]
 [-6.93347439e+07  1.52150561e+08 -6.59086111e+09 ... -9.18956241e+07
  -9.27016543e+07 -9.03803827e+07]
 [ 6.07945530e+08 -1.33281079e+08  3.01513889e+09 ...  6.87911993e+08
   7.35676031e+08  7.15579090e+08]
 ...
 [-2.14827823e+08  3.08682106e+07 -6.56400397e+09 ... -1.01200195e+08
  -9.90416124e+07 -8.36567538e+07]
 [ 4.75618466e+08 -2.81831985e+08  3.23199603e+09 ...  6.99484375e+08
   7.48178602e+08  7.37412582e+08]
 [ 3.27761044e+08 -2.77898391e+08  7.92524603e+09 ...  5.55471680e+08
   5.61563856e+08  5.55804672e+08]]


In [95]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y) # แปลงข้อมูลจาก Frequency 10 Hz ไปเป็น 0 1 2 3 4
# print(y)

[3 3 3 ... 2 2 2]


In [87]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [88]:
# Fitting clasifier to the Training set
# Loading libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

# Instantiate learning model (k = 3)
classifier = KNeighborsClassifier(n_neighbors=3)

# Fitting the model
classifier.fit(x_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
print('Accuracy of our model is equal ' + str(round(accuracy, 2)) + ' %.')
print(cm)

Accuracy of our model is equal 3.05 %.
[[1701 8702 1996  896 1856]
 [9490  215 2002 1155 2013]
 [7164 4648   97  986 2050]
 [9755 2759 1206    2 1242]
 [8445 4951 1305  110  274]]
